In [ ]:
import time
import numpy as np
import nmrglue as ng
import MRAi_2D as ma
import Proc_2D as pr
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':25})

In [ ]:
%%time
spectrum = "Malt_CC.ft2"
Dic , Ref = ng.pipe.read("Spectra/"+spectrum)
dic , ref = pr.pipe_generator.iFT_ID(Dic,Ref)

In [ ]:
%%time
DNN_dirc = "Trained_NN/VD_ID/"
VE = ma.convertors.US_VE(ref,256)
pred = ma.Production.VD_ID(VE, DNN_dirc)
ft1 = np.zeros_like(ref)
ft1[:min(int(pred.shape[0]),int(ft1.shape[0]))] = pred[:min(int(pred.shape[0]),int(ft1.shape[0]))]
_, VD = pr.pipe_generator.FT_ID(dic,ft1)


In [ ]:
%%time

fig, ax = plt.subplots(1,2,figsize=(2*10,10))
cmap = [plt.cm.Greys_r,plt.cm.Reds_r]    #[+,-] contour map (colors to use for contours)
contour = [0.03,1.3,10]         #[contour level start value, scaling factor between contour level]
lim = [1.2, -0.8,26.5, 7.5]                 #[xlim from,xlim to,ylim from,ylim to]

pr.PLOTS.subp(ax[0],Dic,Ref/np.max(Ref), "Ref",contour,cmap,lim)
pr.PLOTS.subp(ax[1],Dic,VD/np.max(VD), "VD",contour,cmap,lim)

ax[1].axes.yaxis.set_visible(False)

plt.subplots_adjust(wspace=0.05, hspace=0.05)